<a href="https://colab.research.google.com/github/amyush/FrequencyDomain_Watermarking/blob/main/watermarking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setting up the environment**

In [1]:
!git clone https://github.com/amyush/FrequencyDomain_Watermarking.git

Cloning into 'FrequencyDomain_Watermarking'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 52 (delta 24), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (52/52), done.


In [2]:
import numpy as np
import cv2
import math
import pywt
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from PIL import Image as im
import math
from scipy.fftpack import dct
from scipy.fftpack import idct
from google.colab.patches import cv2_imshow

# **Custom Functions**

In [81]:
def setParams():
    global alpha 
    alpha = 0.02
    global betaFactor
    betaFactor = 1
    global imgPath
    imgPath = "FrequencyDomain_Watermarking/"
    global factor
    factor = 8

def stringToBinary(a):
  l=[]
  res = ''.join(format(ord(i), '08b') for i in a)
  return res

def calculatePSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                    # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

def apply_dct(image_array):
    # size = (int)(math.sqrt(image_array.size))
    size = len(image_array[0])
    all_subdct = np.empty((size, size))
    for i in range (0, size, 8):
        for j in range (0, size, 8):
            subpixels = image_array[i:i+8, j:j+8]
            subdct = dct(dct(subpixels.T, norm="ortho").T, norm="ortho")
            all_subdct[i:i+8, j:j+8] = subdct

    return all_subdct

def inverse_dct(all_subdct):
    size = len(all_subdct[0])
    all_subidct = np.empty((size, size))
    for i in range (0, size, 8):
        for j in range (0, size, 8):
            subidct = idct(idct(all_subdct[i:i+8, j:j+8].T, norm="ortho").T, norm="ortho")
            all_subidct[i:i+8, j:j+8] = subidct

    return all_subidct

# **Steganography**

## **Hiding the data**

In [85]:
def encodingImage(imageName, key, hiddenImageName):
    setParams();
    binKey = stringToBinary(key)
    lenBinKey = len(binKey)

    # Reading the cover image
    originalImage = cv2.imread(imgPath + imageName)
    print(originalImage.size)

    sizeOrig = (originalImage.shape[0] // 8) * 8
    originalImage = cv2.resize(originalImage, (sizeOrig, sizeOrig))
    print(sizeOrig)
    inputArray = np.array(originalImage[:,:,0], dtype=float)

    # Reading the watermark image
    waterMarkLogo = cv2.imread(imgPath + hiddenImageName)
    sizeWatermark = (waterMarkLogo.shape[0] // 8) * 8
    waterMarkLogo = cv2.resize(waterMarkLogo, (sizeWatermark, sizeWatermark))
    
    waterMarkArray = np.array(waterMarkLogo,dtype=float)

    flatWaterMark = np.zeros(shape = (sizeWatermark,sizeWatermark,3))
    flatWaterMark[:,:,0] = apply_dct(waterMarkArray[:,:,0])
    flatWaterMark[:,:,1] = apply_dct(waterMarkArray[:,:,1])
    flatWaterMark[:,:,2] = apply_dct(waterMarkArray[:,:,2])
    
    flatWaterMark = (flatWaterMark/255).ravel()

    index = -1
    count = 0
    beta = betaFactor

    for i in range(0, sizeOrig, 8):
        for j in range(0, sizeOrig, 8):
            if((int)(binKey[index%lenBinKey]) == 1):
                beta *= -1
            if count < flatWaterMark.size:
                inputArray[i:i+8,j:j+8]=dct(dct(inputArray[i:i+8,j:j+8].T, norm="ortho").T, norm="ortho")
                # inputArray[1][1] += alpha * beta * flatWaterMark[count]
                inputArray[i+1][j+1] = flatWaterMark[count]
                inputArray[i:i+8,j:j+8]=idct(idct(inputArray[i:i+8,j:j+8].T, norm="ortho").T, norm="ortho")
                count += 1

    waterMarkedImage = np.zeros((sizeOrig,sizeOrig,3))
    waterMarkedImage[:,:,0] = inputArray
    waterMarkedImage[:,:,1:] = originalImage[:,:,1:]

    res = cv2.imwrite('waterMarkedImage.jpg', waterMarkedImage)
    return originalImage, waterMarkedImage, sizeWatermark


## **Extracting the watermark image**

In [88]:
def extractImage(waterMarkedImage, key, size):
    setParams();
    binKey = stringToBinary(key)
    lenBinKey = len(binKey)
    
    sizeOrig = waterMarkedImage.shape[0]
    ext_wm=[]
    k=0
    z=0
    
    test_1=waterMarkedImage[:,:,0]

    for i in range(0,sizeOrig,8):
        for j in range(0,sizeOrig,8):
            if(z>=(len(key))):
                z=0
            if(k < size*size*3):
                subdct = test_1[i:i+8,j:j+8]
                subdct =dct(dct(subdct.T, norm="ortho").T, norm="ortho")
                z=z+1
                val=subdct[1][1]
                ext_wm.append(val)
                k=k+1
            else:
                break
        if(k >= size*size*3):
            break

    ext_wm=np.array(ext_wm,dtype=float).reshape((size,size,3))
    logo = np.zeros(shape = (size,size,3))
    logo[:,:,0] = inverse_dct(ext_wm[:,:,0])
    logo[:,:,1] = inverse_dct(ext_wm[:,:,1])
    logo[:,:,2] = inverse_dct(ext_wm[:,:,2])
    
    logo = logo * 255
    logo = logo.astype(np.uint8)

    cv2_imshow(logo)

# **Results**

In [90]:
# coverFileName = input("Enter the cover filename: (eg- cover.jpg or lena.png)")
# key = input("Enter the key:")
# waterMarkImageName = input("Enter the name of the image to be hidden: (eg- iitbbs_logo.jpeg or logo.png)")

coverFileName = 'lena.png'
key = 'Appa'
# hiddenImageName = 'iitbbs_logo.jpeg'
hiddenImageName = 'logo.png'

originalImage, waterMarkedImage, size = encodingImage(coverFileName, key, hiddenImageName)
psnr = calculatePSNR(originalImage, waterMarkedImage)
print('PSNR value = {}'.format(psnr))

# cv2_imshow(originalImage)
# cv2_imshow(waterMarkedImage)

795648
512
PSNR value = 46.292414792287424


In [91]:
extractImage(waterMarkedImage, 'Appa', size)